In [24]:
import pandas as pd
import numpy as np
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
import os

keras.src.models.sequential.Sequential

In [2]:
dataset_slug = 'uciml/pima-indians-diabetes-database'

path = kagglehub.dataset_download(dataset_slug)


100%|██████████| 8.91k/8.91k [00:00<00:00, 12.3MB/s]

Extracting files...


In [5]:
! cd /Users/furkanmelih/.cache/kagglehub/datasets/uciml/pima-indians-diabetes-database/versions/1 && ls

diabetes.csv


In [6]:
df = pd.read_csv(f"{path}/diabetes.csv")

In [7]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [32]:
len(df.columns) -1 , len(df)

(8, 768)

In [8]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [29]:
len(X)

768

In [9]:
X, y

(     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
 0              6      148             72             35        0  33.6   
 1              1       85             66             29        0  26.6   
 2              8      183             64              0        0  23.3   
 3              1       89             66             23       94  28.1   
 4              0      137             40             35      168  43.1   
 ..           ...      ...            ...            ...      ...   ...   
 763           10      101             76             48      180  32.9   
 764            2      122             70             27        0  36.8   
 765            5      121             72             23      112  26.2   
 766            1      126             60              0        0  30.1   
 767            1       93             70             31        0  30.4   
 
      DiabetesPedigreeFunction  Age  
 0                       0.627   50  
 1                    

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
len(X_test), len(X_train)

(154, 614)

In [13]:
len(df)

768

In [17]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
input_features = X_train_scaled.shape[1]

In [21]:
input_features

8

In [25]:
X_train_conv = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_conv = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

print(f"Reshaped training data for Conv1D: {X_train_conv.shape}") # (614, 8, 1)

input_shape_conv = (X_train_conv.shape[1], 1)

model_conv1d = Sequential()

model_conv1d.add(Conv1D(filters=32, kernel_size=2, activation='relu',
                        input_shape=input_shape_conv))

model_conv1d.add(MaxPooling1D(pool_size=2))

model_conv1d.add(Flatten())

model_conv1d.add(Dense(32, activation='relu'))
model_conv1d.add(Dropout(0.2))

model_conv1d.add(Dense(1, activation='sigmoid'))

model_conv1d.summary()

model_conv1d.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

Reshaped training data for Conv1D: (614, 8, 1)


/Users/furkanmelih/personal_projects/okul/yapay_zeka/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 7, 32)          │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 3, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,233 (12.63 KB)

 Trainable params: 3,233 (12.63 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
history_conv1d = model_conv1d.fit(X_train_conv, y_train,
                                  epochs=10,
                                  batch_size=32,
                                  validation_split=0.1,
                                  verbose=0)

print("\n--- Conv1D Model Evaluation ---")
loss_conv, acc_conv = model_conv1d.evaluate(X_test_conv, y_test)
print(f"Conv1D Model Test Accuracy: {acc_conv * 100:.2f}%")


--- Conv1D Model Evaluation ---
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7792 - loss: 0.4821
Conv1D Model Test Accuracy: 77.92%


In [33]:
from sklearn.metrics import classification_report, f1_score

In [36]:
y_pred_probs = model_conv1d.predict(X_test_conv, verbose=0)

In [37]:
y_pred_classes = (y_pred_probs > 0.5).astype(int)

In [38]:
print(classification_report(y_test, y_pred_classes, target_names=['No Diabetes (0)', 'Diabetes (1)']))

                 precision    recall  f1-score   support

No Diabetes (0)       0.81      0.86      0.83        99
   Diabetes (1)       0.71      0.64      0.67        55

       accuracy                           0.78       154
      macro avg       0.76      0.75      0.75       154
   weighted avg       0.78      0.78      0.78       154



In [39]:
f1_weighted = f1_score(y_test, y_pred_classes, average='weighted')

In [40]:
print(f"\nWeighted F1-Score: {f1_weighted:.4f}")


Weighted F1-Score: 0.7761
